In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import numpy as np 
import torch_geometric.nn as pyg_nn 
import torch_geometric.utils as pyg_utils
from torch_geometric.data import Dataset, Data, DataLoader
import torch.optim as optim
import os.path as osp
import scipy.io as sio
from dataset import QUASARDataset
from dual_model import DualModel

In [2]:
dir = '/home/hank/Datasets/QUASAR/small'
dataset = QUASARDataset(dir,num_graphs=100,remove_self_loops=True)
test_dir = '/home/hank/Datasets/QUASAR/mix'
testset = QUASARDataset(test_dir,num_graphs=200,remove_self_loops=True)

Data graph type: 1.
Data graph type: 1.


In [3]:
GNN_TYPE = 'SAGE'
GNN_HIDDEN_DIM = 64
GNN_OUT_DIM = 64
GNN_LAYER = 4
NODE_MODE = 1
DATA_GRAPH_TYPE = 1
DROPOUT = 0
MLP_LAYER = 1
model   = DualModel(node_feature_mode=NODE_MODE,
                     gnn_type=GNN_TYPE,
                     mp_hidden_dim=GNN_HIDDEN_DIM,mp_output_dim=GNN_OUT_DIM,mp_num_layers=GNN_LAYER, 
                     dual_node_mlp_hidden_dim=64,dual_node_mlp_output_dim=10,
                     node_mlp_num_layers=MLP_LAYER,
                     dual_edge_mlp_hidden_dim=64,dual_edge_mlp_output_dim=6,
                     edge_mlp_num_layers=MLP_LAYER, 
                     dropout_rate=DROPOUT,
                     relu_slope=0.1)
model.load_state_dict(torch.load('./models/dual_model_SAGE_4_64_64_1_1_1000_0.0_1.pth'))
model.double()
model.eval()

Model: node_feature_mode = 1, mp_input_dim = 6, relu_slope = 0.1. GNN type: SAGE.


DualModel(
  (mp_convs): ModuleList(
    (0): SAGEConv(6, 64)
    (1): SAGEConv(64, 64)
    (2): SAGEConv(64, 64)
    (3): SAGEConv(64, 64)
    (4): SAGEConv(64, 64)
    (5): SAGEConv(64, 64)
  )
  (dual_node_mlp): ModuleList(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=10, bias=True)
  )
  (dual_edge_mlp): ModuleList(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=6, bias=True)
  )
)

In [5]:
# results on train set
loader = DataLoader(dataset,batch_size=1,shuffle=True)
for batch in loader:
    _, S, Aty = model(batch)
    dual_loss = model.loss(batch,S,Aty)
    print('batch loss: {:.4f}. # nodes: {:d}'.format(
                dual_loss.item(),batch.num_nodes))
    Atyopt = batch.Aty[0]
    Aty    = Aty[0]

batch loss: 0.2543. # nodes: 21
batch loss: 0.1520. # nodes: 21
batch loss: 0.0896. # nodes: 21
batch loss: 0.1071. # nodes: 21
batch loss: 0.1218. # nodes: 21
batch loss: 0.1515. # nodes: 21
batch loss: 0.1801. # nodes: 21
batch loss: 0.1313. # nodes: 21
batch loss: 0.1130. # nodes: 21
batch loss: 0.1289. # nodes: 21
batch loss: 0.1125. # nodes: 21
batch loss: 0.1809. # nodes: 21
batch loss: 0.1432. # nodes: 21
batch loss: 0.1262. # nodes: 21
batch loss: 0.1064. # nodes: 21
batch loss: 0.1617. # nodes: 21
batch loss: 0.1491. # nodes: 21
batch loss: 0.1022. # nodes: 21
batch loss: 0.1252. # nodes: 21
batch loss: 0.1470. # nodes: 21
batch loss: 0.1143. # nodes: 21
batch loss: 0.1412. # nodes: 21
batch loss: 0.1150. # nodes: 21
batch loss: 0.1236. # nodes: 21
batch loss: 0.1390. # nodes: 21
batch loss: 0.1743. # nodes: 21
batch loss: 0.1277. # nodes: 21
batch loss: 0.1102. # nodes: 21
batch loss: 0.1837. # nodes: 21
batch loss: 0.2076. # nodes: 21
batch loss: 0.1360. # nodes: 21
batch lo

In [6]:
# results on test set
loader = DataLoader(testset,batch_size=1,shuffle=True)
for batch in loader:
    _, S, Aty = model(batch)
    dual_loss = model.loss(batch,S,Aty)
    print('batch loss: {:.4f}. # nodes: {:d}'.format(
                dual_loss.item(),batch.num_nodes))
    Atyopt = batch.Aty[0]
    Aty    = Aty[0]

batch loss: 0.6130. # nodes: 31
batch loss: 0.2773. # nodes: 21
batch loss: 0.2339. # nodes: 21
batch loss: 0.4168. # nodes: 21
batch loss: 0.4802. # nodes: 31
batch loss: 0.4362. # nodes: 31
batch loss: 0.4837. # nodes: 31
batch loss: 0.3869. # nodes: 31
batch loss: 0.3570. # nodes: 21
batch loss: 0.5005. # nodes: 31
batch loss: 0.3714. # nodes: 21
batch loss: 0.4667. # nodes: 31
batch loss: 0.3175. # nodes: 21
batch loss: 0.4146. # nodes: 31
batch loss: 0.3681. # nodes: 31
batch loss: 0.5458. # nodes: 31
batch loss: 0.4150. # nodes: 31
batch loss: 0.3896. # nodes: 21
batch loss: 0.4336. # nodes: 31
batch loss: 0.3741. # nodes: 21
batch loss: 0.2590. # nodes: 21
batch loss: 0.4190. # nodes: 21
batch loss: 0.4479. # nodes: 21
batch loss: 0.3304. # nodes: 21
batch loss: 0.3403. # nodes: 31
batch loss: 0.3642. # nodes: 21
batch loss: 0.4077. # nodes: 31
batch loss: 0.2939. # nodes: 21
batch loss: 0.3461. # nodes: 21
batch loss: 0.5160. # nodes: 31
batch loss: 0.3954. # nodes: 31
batch lo